In [3]:
!pip3 install --upgrade google-cloud-documentai
!pip3 install --upgrade google-cloud-storage
!pip3 install --upgrade google-cloud-documentai-toolbox

Defaulting to user installation because normal site-packages is not writeable


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Defaulting to user installation because normal site-packages is not writeable


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Defaulting to user installation because normal site-packages is not writeable


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [123]:
from google.api_core.client_options import ClientOptions
from google.cloud import documentai

PROJECT_ID = "tally-ai-394207"
LOCATION = "us"  # Format is 'us' or 'eu'
PROCESSOR_ID = "e723cbb45d4557ed"  # Create processor in Cloud Console

FILE_PATH = "C:/Users/manish.g/Downloads/DocumentAI (2)/DocumentAI/InvoiceImages/79.jpeg"

#local file in CWD
# Refer to https://cloud.google.com/document-ai/docs/file-types for supported file types (MIME)

#MIME_TYPE = ""

MIME_TYPE = "image/jpeg"

field_mask= "text"

# Instantiates a client
docai_client = documentai.DocumentProcessorServiceClient(
    client_options=ClientOptions(api_endpoint=f"{LOCATION}-documentai.googleapis.com")
)

# The full resource name of the processor

RESOURCE_NAME = docai_client.processor_path(PROJECT_ID, LOCATION, PROCESSOR_ID)

# Read the file into memory
with open(FILE_PATH, "rb") as image:
    image_content = image.read()

# Load Binary Data into Document AI RawDocument Object
raw_document = documentai.RawDocument(content=image_content, mime_type=MIME_TYPE)

# Configure the process request
request = documentai.ProcessRequest(name=RESOURCE_NAME, raw_document=raw_document)

# Use the Document AI client to process the sample form
result = docai_client.process_document(request=request)

document_object = result.document
print("Document processing complete. Below is the output.")



Document processing complete. Below is the output.


In [143]:
import cv2
image_path = FILE_PATH
image = cv2.imread(image_path)
image.shape

(3507, 2480, 3)

In [142]:
def draw_bounding_box( normalized_vertices,entity_type, color=(0, 255, 0), thickness=2):
    height, width, _ = image.shape

    # Convert normalized vertices to pixel coordinates
    x_min, y_min = (
        int(min(normalized_vertex.x for normalized_vertex in normalized_vertices) * width),
        int(min(normalized_vertex.y for normalized_vertex in normalized_vertices) * height)
    )

    x_max, y_max = (
        int(max(normalized_vertex.x for normalized_vertex in normalized_vertices) * width),
        int(max(normalized_vertex.y for normalized_vertex in normalized_vertices) * height)
    )

    # Draw the bounding box as a rectangle
    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2
    text_size = cv2.getTextSize(entity_type, font, font_scale, font_thickness)[0]
    text_color = (255,0,0)

    text_position = (x_min, max(y_min - 5, 0))  # Adjust the vertical position as needed
    cv2.putText(image, entity_type, text_position, font, font_scale, text_color, font_thickness)

In [140]:
import cv2

# ... (your existing code)
# Save the image with bounding boxes
def save_image(input):
    filename=FILE_PATH.split('/')[-1]
    filename
    output_image_path = f"C:/Users/manish.g/Downloads/DocumentAI (2)/DocumentAI/InvoiceImages/{input}_{filename}" 
    # Replace with your desired path
    cv2.imwrite(output_image_path, image)
    
    # Display the saved image path
    print(f"Image with bounding boxes saved at: {output_image_path}")

In [137]:
entities = document_object.entities
entities

[type_: "generic_entities"
page_anchor {
  page_refs {
  }
}
properties {
  text_anchor {
    text_segments {
      end_index: 38
    }
  }
  type_: "organization"
  mention_text: "Pentagon System And Services Pvt. Ltd."
  confidence: 0.971219122
  page_anchor {
    page_refs {
    }
    page_refs {
      bounding_poly {
        normalized_vertices {
          x: 0.313709676
          y: 0.058454521
        }
        normalized_vertices {
          x: 0.63911289
          y: 0.058454521
        }
        normalized_vertices {
          x: 0.63911289
          y: 0.0698602796
        }
        normalized_vertices {
          x: 0.313709676
          y: 0.0698602796
        }
      }
    }
  }
  id: "0"
}
properties {
  text_anchor {
    text_segments {
      start_index: 39
      end_index: 122
    }
  }
  type_: "address"
  mention_text: "444, First Floor, 8th Main, 4th Block, Koramangala,\nBangalore Karnataka - 560034Tel"
  confidence: 0.983675838
  page_anchor {
    page_refs {
    }

In [144]:
c=1
for item in entities[0].properties:
    print(c)
    print(item.type_," :",item.mention_text )
    if "page_anchor" in item:
        if "page_refs" in item.page_anchor:
            
            for page_ref in item.page_anchor.page_refs:
                if "normalized_vertices" in page_ref.bounding_poly:
                    draw_bounding_box(page_ref.bounding_poly.normalized_vertices,item.type)
    
                
    c=c+1
save_image("FormParser_Entities")

1
organization  : Pentagon System And Services Pvt. Ltd.
2
address  : 444, First Floor, 8th Main, 4th Block, Koramangala,
Bangalore Karnataka - 560034Tel
3
phone  : 080 2563 5323
4
organization  : PSSPL
5
email  : Email:pentagonbge@pentagon.co.in
6
id  : 29AAACP0547J1ZC
7
id  : AAACP0547J
8
id  : U72900MH1994PTC081824
9
id  : KR2223/00600
10
date_time  : 19-12-2022
11
date_time  : 7-Dec-22
12
organization  : Karnataka
13
id  : 29
14
id  : -23
15
organization  : Karnataka
16
email  : sreenivasan.n@pentagon.co.in
17
email  : shashidhar.r@pentagon.co.in
18
id  : 0840007413
19
organization  : Warehouse
20
organization  : TALLY SOLUTIONS PVT LTD
21
organization  : TALLY SOLUTIONS PVT LTD
22
address  : AMR Tech Park II
23
address  : AMR Tech Park II, No 23 & 24,
24
address  : Hongasandra, Hosur Main Road,
Bangalore 560 068
25
address  : Hongasandra, Hosur Main Road,
Bangalore 560 068
26
id  : AAACP7879D
27
id  : AAACP7879D
28
id  : 29
29
id  : 560068
30
id  : 29
31
id  : 560068
32
id  : 29AA

In [110]:
fields=document_object.pages[0].form_fields 
fields[0]

field_name {
  text_anchor {
    text_segments {
      start_index: 821
      end_index: 829
    }
    content: "Address\n"
  }
  confidence: 0.791530907
  bounding_poly {
    normalized_vertices {
      x: 0.486177951
      y: 0.344515
    }
    normalized_vertices {
      x: 0.571612
      y: 0.344515
    }
    normalized_vertices {
      x: 0.571612
      y: 0.353415161
    }
    normalized_vertices {
      x: 0.486177951
      y: 0.353415161
    }
  }
  orientation: PAGE_UP
}
field_value {
  text_anchor {
    text_segments {
      start_index: 829
      end_index: 859
    }
    text_segments {
      start_index: 907
      end_index: 955
    }
    text_segments {
      start_index: 973
      end_index: 991
    }
    content: "AMR Tech Park II, No 23 & 24,\nHongasandra, Hosur Main Road,\nBangalore 560 068\nPAN No AAACP7879D\n"
  }
  confidence: 0.791530907
  bounding_poly {
    normalized_vertices {
      x: 0.578629
      y: 0.34673509
    }
    normalized_vertices {
      x: 0.7653

In [147]:
import cv2
image_path = FILE_PATH
image = cv2.imread(image_path)
image.shape

count=1
for field in fields:
    print(count)
    if "field_name" in field and "text_anchor" in field.field_name and "content" in field.field_name.text_anchor:
        print(field.field_name.text_anchor.content)
        draw_bounding_box(field.field_name.bounding_poly.normalized_vertices,str(count))   
    
    if "field_value" in field and "text_anchor" in field.field_value and "content" in field.field_value.text_anchor:
        print(field.field_value.text_anchor.content)
        draw_bounding_box(field.field_value.bounding_poly.normalized_vertices,f"{count}_{field.field_name.text_anchor.content}")   
        
    count = count+1
    
save_image("FormParser_FormField")

1
Address

AMR Tech Park II, No 23 & 24,
Hongasandra, Hosur Main Road,
Bangalore 560 068
PAN No AAACP7879D

2
Ack. Date : 
19-Dec-22

3
Invoice No.
: 
KR2223/00600

4
Invoice Date
: 
19-12-2022

5
State
: 
Karnataka

6
Place of Supply
: 
Karnataka

7
State
: 
Karnataka

8
Address

AMR Tech Park II, No 23 & 24,
Hongasandra, Hosur Main Road,
Bangalore 560 068
PAN No AAACP7879D

9
PO No.

PO-HO/0428/22
-23

10
Ack. No.
: 
112214840308118

11
Name
: 
TALLY SOLUTIONS PVT LTD

12
GSTIN No.
:
29AAACP7879D1Z0

13
GSTIN No.
:
29AAACP7879D1Z0

14
GSTIN No.
:
29AAACP0547J1ZC

15
PAN No.: 
AAACP7879D

16
State
: 
Karnataka

17
Transit Insurance Policy: 
0840007413

18
Warehouse

BANGALORE

19
Back Office

shashidhar.r@pentagon.co.in

20
PAN No.: 
AAACP7879D

21
Total Amount

1,06,531.84

22
State Code: 
29

23
CIN No.
: 
U72900MH1994PTC081824

24
Invoice Total

1,06,531.84

25
Date

7-Dec-22

26
Pin Code : 
560068

27
IRN
: 
8a428ce784bfddbb67c5ae0b51d905ae92175f67d0--
28
Sales Person Name

sreeni